In [12]:
import os
os.chdir("C:/Users/hy138/Desktop/neural_network")

In [13]:
os.getcwd()

'C:\\Users\\hy138\\Desktop\\neural_network'

In [3]:
from efficientAD.efficientADModel import EfficientAD
from common import Config
from PIL import Image
import numpy as np
import cv2
import torch
import os
from sklearn.metrics import f1_score
from PIL import Image

In [6]:
def compute_threshold_on_train(model, train_dir):
    scores = []
    for fname in os.listdir(train_dir):
        if fname.endswith(('.png', '.jpg', '.tiff')):
            img_path = os.path.join(train_dir, fname)
            img = Image.open(img_path).convert('RGB')
            score, _ = model.predict_one_image(img)
            scores.append(score)
    threshold = max(scores)
    print(f"Threshold (max anomaly score on train): {threshold:.4f}")
    return threshold

def evaluate_on_test(model, test_good_dir, test_defect_dir, threshold):
    y_true = []
    y_pred = []

    for fname in os.listdir(test_good_dir):
        if fname.endswith(('.png', '.jpg', '.tiff')):
            img = Image.open(os.path.join(test_good_dir, fname)).convert('RGB')
            score, _ = model.predict_one_image(img)
            y_true.append(0)  
            y_pred.append(1 if score > threshold else 0)

        
    for fname in os.listdir(test_defect_dir):
        if fname.endswith(('.png', '.jpg', '.tiff')):
            img = Image.open(os.path.join(test_defect_dir, fname)).convert('RGB')
            score, _ = model.predict_one_image(img)
            y_true.append(1)  
            y_pred.append(1 if score > threshold else 0)

    f1 = f1_score(y_true, y_pred)
    print(f"F1 Score: {f1:.4f}")
    return f1, y_true, y_pred

def calculateIoU(gtMask, predMask):
    tp = np.logical_and(gtMask == 1, predMask == 1).sum()
    fp = np.logical_and(gtMask == 0, predMask == 1).sum()
    fn = np.logical_and(gtMask == 1, predMask == 0).sum()

    denominator = tp + fp + fn
    if denominator == 0:
        return 1.0  
    return tp / denominator


def evaluate_iou(model, defect_dir, gt_dir, threshold_segmentation):
    iou_scores = []
    
    defect_fnames = sorted(os.listdir(defect_dir))
    gt_fnames = sorted(os.listdir(gt_dir))
    
    for defect_fname, gt_fname in zip(defect_fnames, gt_fnames):
        if defect_fname.endswith(('.png', '.jpg', '.tiff')):
            img_path = os.path.join(defect_dir, defect_fname)
            gt_path = os.path.join(gt_dir, gt_fname)
            
            img = Image.open(img_path).convert('RGB')
            _, heatmap = model.predict_one_image(img)
            
            binary_mask = (heatmap > threshold_segmentation).astype(np.uint8)
            
            gt_mask = np.array(Image.open(gt_path).convert('L'))
            gt_mask = (gt_mask > 127).astype(np.uint8)  # GT mask binarize ediliyor
            
            iou = calculateIoU(gt_mask, binary_mask)
            iou_scores.append(iou)
    
    mean_iou = np.mean(iou_scores)
    print(f"Mean IoU Score: {mean_iou:.4f}")
    return mean_iou


In [5]:
config = Config(model_name="model", output_dir = "./models/efficientad", dataset_path="./wood_dataset/wood/train/", train_steps=10000)

model = EfficientAD(config=config)

model.train()

C:\Users\hy138\Desktop\abc\efficientAD\efficientADModel.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load(self.config.weights, map_loc

Finish


([54.435489654541016,
  31.799819946289062,
  18.173110961914062,
  17.41209602355957,
  16.451335906982422,
  16.90652847290039,
  13.914449691772461,
  19.729537963867188,
  18.078588485717773,
  15.822110176086426,
  13.387723922729492,
  21.7996768951416,
  14.409679412841797,
  12.434385299682617,
  13.998624801635742,
  11.022324562072754,
  22.133790969848633,
  17.613826751708984,
  17.721942901611328,
  18.14628791809082,
  12.989309310913086,
  11.757022857666016,
  21.065122604370117,
  14.309118270874023,
  11.894206047058105,
  14.684185028076172,
  13.833796501159668,
  12.3385009765625,
  21.35242462158203,
  10.512650489807129,
  15.995094299316406,
  15.493163108825684,
  24.5084228515625,
  16.082124710083008,
  20.24159049987793,
  18.440460205078125,
  19.03072166442871,
  13.386758804321289,
  13.98401927947998,
  10.935601234436035,
  10.693441390991211,
  18.595338821411133,
  18.12220573425293,
  26.942392349243164,
  25.193498611450195,
  17.371034622192383,
  

In [14]:
config = Config(model_name="model", pretrain_model_path= "./models/efficientad/model/model.pth")

model = EfficientAD(config=config)
model.initialize_predict()

In [15]:
threshold = compute_threshold_on_train(model, "./wood_dataset/wood/train/good")

Threshold (max anomaly score on train): 0.2425


In [16]:
evaluate_on_test(model, "./wood_dataset/wood/test/good", "./wood_dataset/wood/test/defect", threshold)

F1 Score: 0.9065


(0.9064748201438849,
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [17]:
evaluate_iou(model, "./wood_dataset/wood/test/defect", "./wood_dataset/wood/ground_truth/defect",threshold)

Mean IoU Score: 0.1893


0.1892950111361436